In [4]:
using DataFrames
using Distributions

In [5]:
function generate_data(n1,n2, d)
    # Returns 2 clouds of labelled Gaussian data with different moments
    model1 = MvNormal(-ones(d), .5*eye(d))
    model2 = MvNormal(ones(d), .5*eye(d))
    samples1 = rand(model1, n1)
    samples2 = rand(model2, n2)
    X = DataFrame([transpose(samples1); transpose(samples2)])
    y=DataFrame(Label=ones(size(X,1)))
    y[n1+1:n1+n2,:Label] = -1*ones(n2)
    return X, y
end

generate_data (generic function with 1 method)

In [6]:
function transform_svm_primal(C, X, y)
    d = size(X,2)
    n = size(X,1)
    Q = zeros(d+n, d+n)
    Q[1:d, 1:d] = eye(d)
    p = zeros(d+n)
    p[d+1:d+n] = [C for i in 1:n]
    Y = diagm(y[:Label])
    A = zeros(n, d+n)
    A[1:n, d+1:d+n] = -eye(n)
    A[1:n,1:d] = - Y*Array(X)
    b = -ones(d+n)
    return Q, p, A, b
end

transform_svm_primal (generic function with 1 method)

In [11]:
function transform_svm_dual(C, X, y)
    d = size(X,2)
    n = size(X,1)
    Q = diagm(y[:Label]) * Array(X) * transpose(Array(X)) * diagm(y[:Label])
    p = ones(n)
    A = zeros(2*n, n)
    b = zeros(2*n)
    A[1:n, 1:n] = eye(n)
    A[n+1:2n, 1:n] = -eye(n)
    b[1:n] = C*ones(n)
    return Q, p, A, b
end

transform_svm_dual (generic function with 1 method)

In [12]:
X, y = generate_data(20,20, 15)
transform_svm_dual(.5, X, y)

(
40x40 Array{Float64,2}:
 23.19     14.8388    8.18011  12.8384   …  18.0835  13.5727   11.2164 
 14.8388   15.9189   12.5596   11.9517      16.0907  15.023    10.1851 
  8.18011  12.5596   22.5416    7.03912     19.091    9.62415  14.2965 
 12.8384   11.9517    7.03912  11.9292      12.8895  12.059     7.57035
 20.8688   12.8501   10.2668   11.3295      19.6342  13.2358   12.5833 
 20.5968   19.9305   17.9097   14.2269   …  22.3136  18.0309   15.9675 
  9.48007   6.77833   6.82782   8.64451     13.2905   7.35903   8.23562
 19.3627   17.0606   21.2497   13.2258      26.3768  16.4159   18.4756 
 13.9442   13.5271   15.5553   11.3292      20.2203  11.164    13.4014 
 13.7417   11.1465   14.2002    8.52283     17.8654  10.1823   12.6562 
 10.3185   10.427    15.2179    9.9588   …  16.5492   7.75357  12.1555 
  8.04883   9.97497  12.1103    8.89667     12.0851  10.6541    9.80237
 17.2175   16.7428   14.4977   13.1376      21.459   16.7445   13.0845 
  ⋮                                   

In [ ]:
function Newton(t,x,f,grad,hess)
    
    return x_new, gap
end

In [ ]:
function centering_step(Q,p,A,b,x,t,tol)
    return x_seq

In [ ]:
function barr_method(Q, p, A, b, x_0, mu, tol)
    return x_sol, x_seq